In [ ]:
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import os
import numpy as np
import pandas as pd

# テキストファイルをnumpy配列に読み込み
all_data = np.loadtxt("claim_toiawase_data_python.txt")
#numpy配列をDataFrame型に変換(でーたを見やすくするため)
all_data_pd = pd.DataFrame(all_data)
#DataFrame型の列名を設定
all_data_pd.columns = ["seq", "Imp", "Imp_val", "m_sum_neg", "c_neg", "m_sum_all", "m_sum_nzr", "Mag", "c_nzr", "c_all", "m_max", "s_min", "m_sdv_neg", 
                   "s_sdv_neg", "m_avg_neg", "s_avg_neg", "s_sdv_nzr", "m_sdv_all", "s_sdv_all", "s_avg_nzr", "m_avg_nzr", "s_avg_all", "Sc",
                   "m_avg_all", "m_sum_pos", "c_pos", "m_min", "s_max", "m_sdv_pos", "s_sdv_pos", "m_avg_pos", "s_avg_pos", "m_sdv_nzr"]
print(all_data_pd)   #ImpとImp_valは同じ

# 特徴量の部分のみall_Xに読み込み
all_X = all_data[:9813,3:33]
#ラベル部分をall_yに読み込み
all_y = all_data[:9813, 2]

all_num = np.array(all_data_pd["seq"], dtype= "int64")
# print(all_X)
print(all_num)

         seq  Imp  Imp_val  m_sum_neg  c_neg  m_sum_all  m_sum_nzr  Mag  \
0        1.0  1.0      1.0        1.0    3.0        1.1        1.1  1.2   
1        2.0  1.0      1.0        0.5    1.0        0.5        0.5  0.5   
2        3.0  1.0      1.0        2.1    3.0        2.1        2.1  2.2   
3        4.0  1.0      1.0        1.9    3.0        1.9        1.9  2.0   
4        5.0  1.0      1.0        0.4    1.0        0.5        0.5  0.5   
5        6.0  1.0      1.0        0.5    1.0        0.5        0.5  0.5   
6        7.0  1.0      1.0        3.0    6.0        3.0        3.0  3.2   
7        8.0  1.0      1.0        1.7    3.0        2.6        2.6  2.8   
8        9.0  1.0      1.0        1.1    3.0        1.3        1.3  1.5   
9       10.0  1.0      1.0        1.3    2.0        1.3        1.3  1.3   
10      11.0  1.0      1.0        3.8    5.0        3.8        3.8  4.1   
11      12.0  1.0      1.0        1.0    2.0        1.0        1.0  1.0   
12      13.0  1.0      1.

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold


all_X = all_X[::, :18]

scoring = ["f1", "accuracy", "precision", "recall"]
for c in [0.1,1.0,10.0,100.0,1000.0,10000.0,100000.0]:
    for g in [1.0,0.1,0.01,0.001,0.0001,0.00001]:
        pipe = make_pipeline(StandardScaler(), SVC(C=c, gamma=g, kernel="rbf", random_state=1))
        kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=2)
        scores = cross_validate(pipe, all_X, all_y, scoring=scoring, cv=kf)
        print("C:%s      gamma:%s" %(c,g))
        for key,value in scores.items():
            print("{}:{:.4f}+/-{:.4f}".format(key, value.mean(), value.std()))
        print("\n")
        
        

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

for f in [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12 ,13, 14, 15, 16, 17, 18]:
    all_X = all_X[::, :f]

    for c in [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]:
        for g in [1.0, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001]:
            i = 1
    #       十回分のそれぞれのスコアを格納する空のlistを作成
            accuracy_train = []
            f1_train = []
            precision_train = []
            recall_train = []

            accuracy_test = []
            f1_test = []
            precision_test = []
            recall_test = []
    #       パイプラインの作成
            pipe = make_pipeline(StandardScaler(), SVC(C=c, gamma=g, kernel="rbf", random_state=1))
            #層化10-fold交差検証の開始
            kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=2)
            for train_index, test_index in kf.split(all_X, all_y):
                train_X, test_X = all_X[train_index], all_X[test_index]
                train_y, test_y = all_y[train_index], all_y[test_index]
                train_num, test_num = all_num[train_index], all_num[test_index]
    #             訓練データのラベル1のsmote前のデータ数の出力
    #             print("train_X(label=1): %s" %train_X[train_y == 1].shape[0])
    #             print("train_X(label=0): %s" %train_X[train_y == 0].shape[0])
                pipe.fit(train_X, train_y)
                pre_label_train = pipe.predict(train_X)
                accuracy_train.append(accuracy_score(train_y, pre_label_train))
                f1_train.append(f1_score(train_y, pre_label_train))
                precision_train.append(precision_score(train_y, pre_label_train))
                recall_train.append(recall_score(train_y, pre_label_train))

                pre_label_test = pipe.predict(test_X)
    #             print(pre_label_test)
                accuracy_test.append(accuracy_score(test_y, pre_label_test))
                f1_test.append(f1_score(test_y, pre_label_test))
                precision_test.append(precision_score(test_y, pre_label_test))
                recall_test.append(recall_score(test_y, pre_label_test))
    #             print(test_num)
                V_test_num = test_num.reshape(len(test_num), 1)
                V_pre_label_test = pre_label_test.reshape(len(pre_label_test), 1)
    #             print(V_test_num)
    #             print(V_pre_label_test)
                num_and_y = np.hstack((V_test_num, V_pre_label_test))

                print(i)
                np.savetxt("%d_direct_num_and_prelabel.csv" %i, num_and_y, fmt="%3.f", delimiter="\t", header="文書番号\t予測ラベル")
                i = i + 1

            #十回のスコアの平均と標準偏差の出力
            print("\n")
            print("C:%s      gamma:%s" %(c,g))
            print("accuracy_train: %.4f +- %.4f" %(np.mean(accuracy_train), np.std(accuracy_train)))
            print("f1_train: %.4f +- %.4f" %(np.mean(f1_train), np.std(f1_train)))
            print("precision_train: %.4f +- %.4f" %(np.mean(precision_train), np.std(precision_train)))
            print("recall_train: %.4f +- %.4f" %(np.mean(recall_train), np.std(recall_train)))

            print("accuracy_test: %.4f +- %.4f" %(np.mean(accuracy_test), np.std(accuracy_test)))
            print("f1_test: %.4f +- %.4f" %(np.mean(f1_test), np.std(f1_test)))
            print("precision_test: %.4f +- %.4f" %(np.mean(precision_test), np.std(precision_test)))
            print("recall_test: %.4f +- %.4f" %(np.mean(recall_test), np.std(recall_test)))
            print("list_shape: %s" %len(accuracy_train))
            print("\n")

1
2
3
4
5
6
7
8
9
10


C:0.1      gamma:1.0
accuracy_train: 0.8818 +- 0.0009
f1_train: 0.4330 +- 0.0067
precision_train: 0.7035 +- 0.0067
recall_train: 0.3129 +- 0.0067
accuracy_test: 0.8813 +- 0.0058
f1_test: 0.4297 +- 0.0306
precision_test: 0.7043 +- 0.0567
recall_test: 0.3107 +- 0.0311
list_shape: 10


